In [1]:
import os
import cv2
import numpy as np

In [ ]:
# 이미지 회전
def rotate_image(image, angle):
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated

# 이미지 반전 / flip_code: 1 (좌우), 0 (상하), -1 (상하좌우)
def flip_image(image, flip_code):
    flipped = cv2.flip(image, flip_code)
    return flipped

# 이미지 확대/축소
def resize_image(image, scale_factor):
    h, w = image.shape[:2]
    resized = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))
    return resized

# 밝기 조절
def adjust_brightness(image, value):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v, value)
    v = np.clip(v, 0, 255)
    final_hsv = cv2.merge((h, s, v))
    brightened = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return brightened

# 이미지 노이즈 추가
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 15
    gauss = np.random.normal(mean, sigma, (row, col, ch)).astype('uint8')
    noisy_image = cv2.add(image, gauss)
    return noisy_image

# 색상 변환
def change_color(image, color_scale):
    transformed_image = cv2.applyColorMap(image, color_scale)
    return transformed_image

# Fancy PCA
def fancy_pca(image, alpha_std=0.1):
    # 이미지 크기와 채널 분리
    orig_image = image.astype("float32")
    orig_shape = orig_image.shape
    reshaped_image = orig_image.reshape(-1, 3)

    # 채널별 평균값 계산 후 제거
    mean = np.mean(reshaped_image, axis=0)
    centered_image = reshaped_image - mean

    # 공분산 행렬 계산
    cov = np.cov(centered_image, rowvar=False)

    # 고유값과 고유벡터 계산
    eig_vals, eig_vecs = np.linalg.eigh(cov)

    # 랜덤 노이즈 생성
    noise = np.random.normal(0, alpha_std, size=(3,))
    delta = eig_vecs @ (noise * eig_vals**0.5)

    # 색상 조정
    pca_image = reshaped_image + delta
    pca_image = np.clip(pca_image + mean, 0, 255).reshape(orig_shape).astype("uint8")
    return pca_image


# Gaussian Blur
def blur_image(image):
    kernel_size = (25, 25)
    blurred = cv2.GaussianBlur(image, kernel_size, 0)
    return blurred

def augment_images(image):
    augmented_images = []
    augmented_images.append(("rotated", rotate_image(image, 15))) # 회전
    augmented_images.append(("resized", resize_image(image, 1.2))) # 확대
    augmented_images.append(("flipped", flip_image(image, 1))) # 좌우 뒤집기
    augmented_images.append(("brightness", adjust_brightness(image, 10))) # 밝기 조절
    augmented_images.append(("noised", add_noise(image))) # 노이즈 추가
    augmented_images.append(("colored", change_color(image, cv2.COLORMAP_DEEPGREEN))) # 색상 변환 (예: COLORMAP_JET)
    augmented_images.append(("fancypca", fancy_pca(image)))
    augmented_images.append(("blurred", blur_image(image)))
    return augmented_images

In [ ]:
# 원본 디렉토리와 저장 디렉토리 지정
input_dir = 'input'   # 원본 이미지 폴더
output_dir = 'output'  # 증강된 이미지 저장 폴더
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    img_path = os.path.join(input_dir, filename)

    if os.path.isfile(img_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        image = cv2.imread(img_path)

        # 증강된 이미지 리스트 받기
        augmented_images = augment_images(image)
        
        # 각 증강 이미지 저장
        for aug_type, aug_image in augmented_images:
            # "파일명_증강유형.확장자" 형식으로 저장
            base_name, ext = os.path.splitext(filename)
            save_path = os.path.join(output_dir, f"{base_name}_{aug_type}{ext}")
            cv2.imwrite(save_path, aug_image)